In [ ]:
!rm -rf xtensor xtensor-io xtl

In [ ]:
!git clone --single-branch --branch zarray https://github.com/davidbrochart/xtensor.git

In [ ]:
!git clone https://github.com/xtensor-stack/xtensor-io.git

In [ ]:
!git clone https://github.com/xtensor-stack/xtl.git

In [ ]:
#pragma cling add_include_path("xtensor/include")
#pragma cling add_include_path("xtensor-io/include")
#pragma cling add_include_path("xtl/include")
#pragma cling add_include_path("../include")

In [ ]:
#include "xtensor/xview.hpp"
#include "xtensor/xchunked_array.hpp"
#include "xtensor/xchunk_store_manager.hpp"
#include "xtensor/xfile_array.hpp"
#include "xtensor/xdisk_io_handler.hpp"
#include "xtensor-io/xio_binary.hpp"
#include "xtensor-zarr/xzarr_hierarchy.hpp"
#include "xtensor-zarr/xzarr_file_system_store.hpp"

In [ ]:
using namespace xt;

In [ ]:
!mkdir -p chunks1 chunks2 chunks3

In [ ]:
std::vector<size_t> shape = {4, 4};
std::vector<size_t> chunk_shape = {2, 2};

using array = xchunked_array<xchunk_store_manager<xfile_array<double, xdisk_io_handler<xio_binary_config>>>>;
array a1(shape, chunk_shape);
array a2(shape, chunk_shape);
array a3(shape, chunk_shape);
a1.chunks().set_directory("chunks1");
a2.chunks().set_directory("chunks2");
a3.chunks().set_directory("chunks3");
a1(0, 0) = 1;
a2(2, 2) = 1;
view(a3, all(), all()) = view(a1, all(), all()) + view(a2, all(), all());
std::cout << a1 << std::endl;
std::cout << a2 << std::endl;
std::cout << a3 << std::endl;

In [ ]:
!ls chunks1 chunks2 chunks3

In [ ]:
std::vector<size_t> shape = {4, 4};
std::vector<size_t> chunk_shape = {2, 2};
nlohmann::json attrs = {{"question", "life"}, {"answer", 42}};
xzarr_file_system_store s("h_xtensor.zr3");
auto h = create_zarr_hierarchy(s);
zarray z = h.create_array("/arthur/dent", shape, chunk_shape, "<f8", '/', xio_binary_config(), attrs);

In [ ]:
!find h_xtensor.zr3

In [ ]:
!cat h_xtensor.zr3/meta/root/arthur/dent.array